## Import Data

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense
from keras.models import Sequential

In [ ]:
## import the data
DATADIR = ".//Data"
CATEGORIES = ["solid", "wet"]
for category in CATEGORIES:
# create path to cars and trucks
    path = os.path.join(DATADIR,category)  
# iterate over each image per cars and trucks
    for img in os.listdir(path):
# convert to array
        img_array = cv2.imread(os.path.join(path,img))

In [ ]:
## create training data
IMG_SIZE = 80
training_data = []

In [ ]:
## create traing data
def create_training_data():
    for category in CATEGORIES:  # do cars and trucks

        path = os.path.join(DATADIR,category)  # create path to cars and trucks
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=solid 1=wet

        for img in tqdm(os.listdir(path)):  # iterate over each image cars and trucks
            try:
        # convert to array
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)  
                # resize to normalize data size
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass

create_training_data()

In [ ]:
## create features and labels
import random
random.shuffle(training_data)

X_wet_solid = []
y_wet_solid = []

for features,label in training_data:
    X_wet_solid.append(features)
    y_wet_solid.append(label)

X_wet_solid = np.array(X_wet_solid).reshape(-1,
                        IMG_SIZE, IMG_SIZE,
                        1)

In [ ]:
##write the pickle file
import pickle

pickle_out = open("X_wet_solid.pickle","wb")
pickle.dump(X_wet_solid, pickle_out)
pickle_out.close()

pickle_out = open("y_wet_solid.pickle","wb")
pickle.dump(y_wet_solid, pickle_out)
pickle_out.close()

In [1]:
##divide training and testing dataset
import pickle
import numpy

# -------------Load X--------------
pickle_in = open("X_wet_solid.pickle","rb") 
train_images = pickle.load(pickle_in)
# Split part of X for testing purpose
test_images = train_images[int(0.9 * len(train_images)):]
train_images = train_images[:int(0.9 * len(train_images))]

# -------------Load y--------------
pickle_in = open("y_wet_solid.pickle","rb") 
train_labels = pickle.load(pickle_in)
#Split part of Y for testing purpose
test_labels = train_labels[int(0.9 * len(train_labels)):]
train_labels = numpy.asarray(train_labels[:int(0.9 * len(train_labels))])

# convert list to ndarray, and reshape the labels
test_labels = numpy.array(test_labels).reshape(len(test_labels),1)
train_labels = numpy.array(train_labels).reshape(len(train_labels),1)

In [2]:
print(f"train_images - Type: {type(train_images)}, Shape {train_images.shape}")
print(f"test_images - Type: {type(test_images)}, Shape {test_images.shape}")

train_images - Type: <class 'numpy.ndarray'>, Shape (8998, 80, 80, 1)
test_images - Type: <class 'numpy.ndarray'>, Shape (1000, 80, 80, 1)


In [3]:
train_images

array([[[[255],
         [255],
         [255],
         ...,
         [255],
         [255],
         [255]],

        [[255],
         [255],
         [255],
         ...,
         [255],
         [255],
         [255]],

        [[255],
         [255],
         [255],
         ...,
         [255],
         [255],
         [255]],

        ...,

        [[255],
         [255],
         [250],
         ...,
         [255],
         [255],
         [255]],

        [[255],
         [255],
         [255],
         ...,
         [255],
         [255],
         [255]],

        [[255],
         [255],
         [252],
         ...,
         [255],
         [255],
         [255]]],


       [[[221],
         [219],
         [216],
         ...,
         [130],
         [130],
         [128]],

        [[223],
         [221],
         [218],
         ...,
         [130],
         [130],
         [130]],

        [[221],
         [220],
         [220],
         ...,
         [130],
         [

## Build model

In [4]:
## import package
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [5]:
## set the randomseed
from DL_tools import fixRandomSeed as frs
frs.fixRandomSeed()

fixRandomSeed in action


In [6]:
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [9]:
model_wet_solid = Sequential([
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(train_images.shape[1:])),
    MaxPooling2D(pool_size=2),

    Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),

    Flatten(),

    Dense(64, activation='relu'),

    Dense(2, activation='sigmoid')
])

In [10]:
##Compile the model
model_wet_solid.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
history = model_wet_solid.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

Epoch 1/10
282/282 [==============================] - 47s 167ms/step - loss: 0.1928 - accuracy: 0.9238 - val_loss: 0.1737 - val_accuracy: 0.9310
Epoch 2/10
282/282 [==============================] - 48s 169ms/step - loss: 0.1440 - accuracy: 0.9430 - val_loss: 0.1425 - val_accuracy: 0.9470
Epoch 3/10
282/282 [==============================] - 47s 166ms/step - loss: 0.1089 - accuracy: 0.9590 - val_loss: 0.1355 - val_accuracy: 0.9490
Epoch 4/10
282/282 [==============================] - 51s 180ms/step - loss: 0.0858 - accuracy: 0.9690 - val_loss: 0.1287 - val_accuracy: 0.9520
Epoch 5/10
282/282 [==============================] - 50s 175ms/step - loss: 0.0736 - accuracy: 0.9725 - val_loss: 0.1027 - val_accuracy: 0.9630
Epoch 6/10
282/282 [==============================] - 46s 164ms/step - loss: 0.0540 - accuracy: 0.9796 - val_loss: 0.1051 - val_accuracy: 0.9670
Epoch 7/10
282/282 [==============================] - 46s 165ms/step - loss: 0.0412 - accuracy: 0.9848 - val_loss: 0.1085 - val_ac

In [12]:
# Evaluate the model
test_loss, test_acc = model_wet_solid.evaluate(test_images,  test_labels, verbose=2)
print(f"Loss = {round(test_loss,2)}, Acc = {round(test_acc,2)}")

32/32 - 1s - loss: 0.2457 - accuracy: 0.8950
Loss = 0.25, Acc = 0.89


## Test Data

In [ ]:
import cv2

#打开摄像头
cap = cv2.VideoCapture(0)

while(1):
    # 读取图片
    ret, frame = cap.read()
    # 展示获取的图片
    cv2.imshow("capture", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        # 图片写入文件
        cv2.imwrite("photo.jpg", frame)
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
photo = cv2.imread("photo.jpg", cv2.IMREAD_GRAYSCALE)
new_photo = cv2.resize(photo, (IMG_SIZE, IMG_SIZE))
new_photo = new_photo.reshape(-1,IMG_SIZE, IMG_SIZE, 1)
new_photo = new_photo / 255.0

In [ ]:
print("Predict result is:",CATEGORIES[model_solid.predict_classes(new_photo)[0]])